# Resize Image System

## Introduction

As my job role requires working with pictures, I am bound to resize them to a specific size so we can maintain the wonderful design and keep the organization pattern of items added to our website. I found myself having to do this process online, which takes time. Then, I got the idea to build a small little app using **customtkinter** and **Pillow** for image processing in Python. This way, I can just paste the path of the picture, and it will be resized and saved automatically at the same path.

However, I realized that this still takes a little more time than doing the whole process online, although it is more convenient. So, I thought about finding a way to automate the resizing of any picture added to a specific folder. This way, I wouldn't need to copy the image path, paste it into the app, and click the resize button every time I needed to resize a picture. The **watchdog** package helped with this exactly. It keeps an eye on the specific folder I'm working with, and whenever I add a new picture, it resizes and saves it automatically. It might not seem like a big thing, but it has saved me time, which matters

## First Version 

In this version I tried to create a simple GUI application using **CustomTkinter** to resize and pad images. Here is an explanation of the idea and the script.

The following cell concerns the Imports:

**customtkinter** : that's a custom module for Tkinter to create modern looking GUI applications. <br />
**PIL.Image** : a module from the python Imaging Library (Pillow) to handle image processing taks.

In [1]:
import customtkinter 
from PIL import Image

As regards to the appearance and theme settings, I set the appearance mode to "light" using **customtkinter.set_appearance_mode("light")**, and the default color theme to "dark-blue" using **customtkinter.set_default_color_theme("dark-blue")**. <br />
Also I initialized the main window and set its size to 500 x 350 px.

In [2]:
customtkinter.set_appearance_mode("light")  
customtkinter.set_default_color_theme("dark-blue")  

root = customtkinter.CTk()  
root.geometry("500x350") 

In the following cell, I tried to build the **resize_and_pad_image** Function which will be trigged when the user clicks the "Resize" Button. The function retrieves the file path entered by the user.<br />

**input_path = entry1.get()**: Gets the file path entered by the user from entry1. <br/>
**input_path = input_path.strip('"')**: Removes any leading or trailing quotes from the file path. I used the copy the image path using Ctrl+Shift+C or simply you can click right on your image then select "Copy as Path". The copied path always appears so **"C:\Users\Desktop\Image Path"**. <br/>
**size = (500, 500)**: Defines the target size for the resized image.<br/>

For the **image processing**, if the image has an alpha channel (transparent background), it processes the image accordingly. However if it doesn't have an alpha channel, it processes the image assuming a white background. <br/>

The image is resized while maintaining its aspect ratio, then padded to fit 500 x 500 pixel canvas. Finally, the processed image is saved back to the original file path.<br/>

At the end I tried to add **Error Handling** to handle file not found errors and other exceptions, updating the feedback label with the appropriate message. I ensure clearing the entry field after each processing.

In [3]:
def resize_and_pad_image():
    input_path = entry1.get()  
    input_path = input_path.strip('"')  
    size = (500, 500)  

    try:
        # Open the input image
        image = Image.open(input_path)

        # Check if the image has an alpha channel
        has_alpha = image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info)

        if has_alpha:
            # Process the image with a transparent background
            image = image.convert("RGBA")
            # Calculate the ratio
            ratio = min(size[0] / image.width, size[1] / image.height)
            # Calculate the new size preserving the aspect ratio
            new_size = (int(image.width * ratio), int(image.height * ratio))
            # Resize the image
            image = image.resize(new_size, Image.LANCZOS)
            # Create a new image with a transparent background
            new_image = Image.new("RGBA", size, (0, 0, 0, 0))
            # Calculate the position to paste the resized image onto the new image
            paste_position = ((size[0] - new_size[0]) // 2, (size[1] - new_size[1]) // 2)
            # Paste the resized image onto the new image
            new_image.paste(image, paste_position, image)

        else:
            # Process the image with a white background
            image = image.convert("RGB")
            # Calculate the ratio
            ratio = min(size[0] / image.width, size[1] / image.height)
            # Calculate the new size preserving the aspect ratio
            new_size = (int(image.width * ratio), int(image.height * ratio))
            # Resize the image
            image = image.resize(new_size, Image.LANCZOS)
            # Create a new image with a white background
            new_image = Image.new("RGB", size, (255, 255, 255))
            # Calculate the position to paste the resized image onto the new image
            paste_position = ((size[0] - new_size[0]) // 2, (size[1] - new_size[1]) // 2)
            # Paste the resized image onto the new image
            new_image.paste(image, paste_position)

        # Save the new image, overwriting the original file
        new_image.save(input_path)
        label3.configure(text="Image resized and saved")

    except FileNotFoundError:
        label3.configure(text="The specified file was not found.")
    except Exception as e:
        label3.configure(text=f"An error occurred: {e}")

    # Clear the entry1 content
    entry1.delete(0, customtkinter.END)

In the next cell, I defined the **Frame** widget to hold the GUI components, **Labels** to display static text and feedback messages, **Entry** widget for the user to input the image path, and **Button** to trigger the "resize_and_pad_image" function.<br/>

We find also the even loop **root.mainloop()** that keeps the GUI running until the user closes the window.

In [4]:
frame = customtkinter.CTkFrame(master=root)
frame.pack(pady=20, padx=60, fill="both", expand=True)

label1 = customtkinter.CTkLabel(master=frame, text='Resize Image System')
label1.pack(pady=12, padx=10)

entry1 = customtkinter.CTkEntry(master=frame, placeholder_text="Enter Image Path", width=200)
root.update()
entry1.focus_set()
entry1.pack(pady=12, padx=10)

button = customtkinter.CTkButton(master=frame, text="Resize", command=resize_and_pad_image)
button.pack(pady=12, padx=10)

label2 = customtkinter.CTkLabel(master=frame, text='Feedback :')
label2.pack(pady=12, padx=10)

label3 = customtkinter.CTkLabel(master=frame, text='')
label3.pack(pady=12, padx=10)

root.mainloop()

## Second Version

In this version, I just updated the user§friendly GUI app that allowed me to monitor a specified folder for new image files. When new images are added, they are automatically resized and padded to fit a 500 x 500 pixel canvas, handling both transparent and non-transparent backgrounds. The start and stop functionality for folder monitoring provided me with control over the process.

In [1]:
import time
import os
import customtkinter  # Import the CustomTkinter module
from PIL import Image
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

customtkinter.set_appearance_mode("light")  # Set the appearance mode to light
customtkinter.set_default_color_theme("dark-blue")  # Set the default color theme to dark-blue

root = customtkinter.CTk()  # Initialize the main window
root.geometry("500x350")  # Set the window size

observer = None  # Global observer variable


class ImageHandler(FileSystemEventHandler):
    def __init__(self, size=(500, 500)):
        self.size = size

    def on_created(self, event):
        if not event.is_directory and event.src_path.lower().endswith(('png', 'jpg', 'jpeg')):
            self.resize_and_pad_image(event.src_path)
            root.focus_force()

    def resize_and_pad_image(self, input_path, retries=5, delay=0.5):
        for attempt in range(retries):
            try:
                # Open the input image
                image = Image.open(input_path)

                if image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info):
                    # Image has transparency
                    image = image.convert("RGBA")
                    # Calculate the ratio
                    ratio = min(self.size[0] / image.width, self.size[1] / image.height)
                    # Calculate the new size preserving the aspect ratio
                    new_size = (int(image.width * ratio), int(image.height * ratio))
                    # Resize the image
                    image = image.resize(new_size, Image.LANCZOS)
                    # Create a new image with a transparent background
                    new_image = Image.new("RGBA", self.size, (0, 0, 0, 0))
                    # Calculate the position to paste the resized image onto the new image
                    paste_position = ((self.size[0] - new_size[0]) // 2, (self.size[1] - new_size[1]) // 2)
                    # Paste the resized image onto the new image
                    new_image.paste(image, paste_position, image)

                else:
                    # Image does not have transparency, assume it has a white background
                    image = image.convert("RGB")
                    # Calculate the ratio
                    ratio = min(self.size[0] / image.width, self.size[1] / image.height)
                    # Calculate the new size preserving the aspect ratio
                    new_size = (int(image.width * ratio), int(image.height * ratio))
                    # Resize the image
                    image = image.resize(new_size, Image.LANCZOS)
                    # Create a new image with a white background
                    new_image = Image.new("RGB", self.size, (255, 255, 255))
                    # Calculate the position to paste the resized image onto the new image
                    paste_position = ((self.size[0] - new_size[0]) // 2, (self.size[1] - new_size[1]) // 2)
                    # Paste the resized image onto the new image
                    new_image.paste(image, paste_position)

                # Save the new image, overwriting the original file
                new_image.save(input_path)
                label3.configure(text="Image resized and saved")
                root.focus_force()
                return

            except FileNotFoundError:
                label3.configure(text="The specified file was not found. Retrying...")
                root.focus_force()
                time.sleep(delay)
            except Exception as e:
                label3.configure(text=f"An error occurred: {e}")
                root.focus_force()
                return


def start_watching():
    global observer
    folder_path = entry1.get()  # Get the folder path from entry1
    folder_path = folder_path.strip('"')
    if not os.path.isdir(folder_path):
        label3.configure(text="The specified path is not a valid folder.")
        root.focus_force()
        return

    event_handler = ImageHandler()
    observer = Observer()
    observer.schedule(event_handler, path=folder_path, recursive=False)
    observer.start()
    label3.configure(text=f"Watching folder: {folder_path}")
    root.focus_force()


def stop_watching():
    global observer
    if observer:
        observer.stop()
        observer.join()
        observer = None
        label3.configure(text="Stopped watching.")
        root.focus_force()


frame = customtkinter.CTkFrame(master=root)
frame.pack(pady=20, padx=60, fill="both", expand=True)

label1 = customtkinter.CTkLabel(master=frame, text='Resize Image System')
label1.pack(pady=12, padx=10)

entry1 = customtkinter.CTkEntry(master=frame, placeholder_text="Enter Folder Path", width=200)
root.update()
entry1.focus_set()
entry1.pack(pady=12, padx=10)

button_start = customtkinter.CTkButton(master=frame, text="Start Watching", command=start_watching)
button_start.pack(pady=12, padx=10)

button_stop = customtkinter.CTkButton(master=frame, text="Stop Watching", command=stop_watching)
button_stop.pack(pady=12, padx=10)

label2 = customtkinter.CTkLabel(master=frame, text='Feedback :')
label2.pack(pady=12, padx=10)

label3 = customtkinter.CTkLabel(master=frame, text='')
label3.pack(pady=12, padx=10)

root.mainloop()

## Conclusion

I am fully aware that the provided app is not really that big thing, but it actually does the work, which matters to me. I've always had the idea of automating boring tasks and putting my brilliant ideas into effect. I can not say that it's always easy and feasible but it certainly helps and saves time.